In [ ]:
!pip install -q -U openai "pymongo[srv]"

In [ ]:
from openai import OpenAI
openai_client = OpenAI()

def get_embedding(texts, model="text-embedding-3-small"):
    texts = map(lambda t: t.replace("\n", " "), texts)
    return openai_client.embeddings.create(input=texts, model=model, dimensions=512).data



In [ ]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi
import os
mongo_client = MongoClient(os.environ["MONGODB_CONNECTION_STRING"], server_api=ServerApi('1'))
try:
    mongo_client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
    
except Exception as e:
    print(e)

In [ ]:
db = mongo_client["amazon_food_reviews"]
afr_collection= db["amazon_food_reviews"]

link to reviews https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

In [ ]:
#

# import random

# import csv
# with open('Reviews.csv', 'r') as f:
#      i = 0
#      arr = []        
#      arr_size = 0   
#      reader = csv.DictReader(f)
#      sample = set(random.sample([x for x in range(0,560000)], 10000))
#      print("Sample length: ", len(sample))     
#      for r in reader:
#          i += 1
#          if i % 10000 == 0:
#              print("lines read: " , i)
#          if i in sample:
#              arr.append(r)
#              arr_size += 1
#              if arr_size >= 100:
#                  arr_emb = get_embedding([r["Summary"] + ' ' +r["Text"] for r in arr])
#                  arr = [ a | {"summary_text_embedding": arr_emb[idx].embedding} for idx, a in enumerate(arr)]
#                  afr_collection.insert_many(arr)
#                  arr = []
#                  arr_size = 0
              
             
             

In [ ]:
afr_collection.count_documents({})

In [ ]:
#Could only set the index at Atlas web interface creating a free cluster

# from pymongo.operations import SearchIndexModel

# search_model = SearchIndexModel(
#     definition={
#         "fields": [
#             {
#               "type": "vector",
#               "path": "summary_text_embedding",
#               "numDimensions": 512,
#               "similarity": "dotProduct"
#             },
#             {
#               "type": "filter",
#               "path": "ProductId"
#             },
#             {
#               "type": "filter",
#               "path": "Score"
#             },
#             {
#               "type": "filter",
#               "path": "Time"
#             }
             
#        ]
#    },
#    name="summary_text_embedding_search_vector_index",
#    type="vectorSearch" 
# )

# afr_collection.create_search_index(search_model)

In [ ]:
query_embedding = get_embedding(["pets food"])[0].embedding
print(len(query_embedding))



In [ ]:
pipeline = [
  {
    '$vectorSearch': {
      'index': 'summary_text_embedding_vector_index', 
      'path': 'summary_text_embedding', 
      'queryVector': query_embedding, 
      'numCandidates': 150, 
      'limit': 3,
      "filter": {'Score': '5'}  
    }
  }, {
    '$project': {
      '_id': 0, 
      'Summary': 1, 
      'Text': 1, 
      'score': {
        '$meta': 'vectorSearchScore'
      }
    }
  }
]

In [ ]:
result = afr_collection.aggregate(pipeline)
# print results
for i in result:
    print(i)

### The search is returning two equals text. Let's use max marginal relevance to get results with some more diversification avoiding repetitions

In [ ]:
!pip install -q langchain_community langchain-mongodb tiktoken


In [ ]:
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.embeddings.openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512)
vectorstore = MongoDBAtlasVectorSearch(
    afr_collection,
    embedding_model,
    text_key='Text',
    embedding_key='summary_text_embedding',
    index_name='summary_text_embedding_vector_index',
    relevance_score_fn='dotProduct',
    
    
)

In [ ]:
query = "pets food"

In [ ]:
#If you don't project Text(our page content), score and summary_text_embedding(our embeddings field)
# this is not going to work. All other fields are not mandatory in projection
responses = vectorstore.max_marginal_relevance_search(
    query,
    k=3,
    fetch_k=20,
    lambda_mult=0.5,
    pre_filter={'Score': '5'},
    post_filter_pipeline=[
        {'$project': {'_id': 0, 'Summary': 1, 'Text': 1, 'score': 1, 'summary_text_embedding': 1}}
    ]
    
)
    

In [ ]:
for r in responses:
    #print(r)
    print(r.metadata['Summary'],' | ', r.page_content, '\n')